In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-187962
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-187962


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


cluster_name = "compute-cluster"
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Existing Compute using {cluster_name}")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)

Existing Compute using compute-cluster


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.1, 2.0),
    "--max_iter": choice(50, 100, 150, 200)
})
# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(  script="train.py",
                        source_directory=".",
                        environment=sklearn_env,
                        compute_target=compute_target)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    run_config=src,
                                    policy=policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_concurrent_runs=4,
                                    max_total_runs=20
                                    )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
exp_run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(exp_run).show()
exp_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = exp_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_file_names = best_run.get_file_names()
print('Metrics: ', best_run_metrics)
print("\n\nBest run file names :", best_run_file_names)
print('\n\nRun Id: {} Accuracy: {}'.format(best_run.id, best_run_metrics['Accuracy']))
model_path = "./outputs/{}_model.joblib".format(best_run.id)
final_model = best_run.register_model(model_name = 'hyperdrive_model',model_path = model_path)


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

In [4]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
final_data = pd.concat([x, y], axis=1)
print(final_data.head())

train_data, test_data = train_test_split(final_data, test_size=0.2)
ds = ws.get_default_datastore()

automl_dataset = Dataset.Tabular.register_pandas_dataframe(train_data, ds, 'automl-dataset')

   age  marital  default  housing  loan  month  day_of_week  duration  \
0   57        1        0        0     1      5            1       371   
1   55        1        0        1     0      5            4       285   
2   33        1        0        0     0      5            5        52   
3   36        1        0        0     0      6            5       355   
4   27        1        0        1     0      7            5       189   

   campaign  pdays  ...  contact_telephone  education_basic.4y  \
0         1    999  ...                  0                   0   
1         2    999  ...                  1                   0   
2         1    999  ...                  0                   0   
3         4    999  ...                  1                   0   
4         2    999  ...                  0                   0   

   education_basic.6y  education_basic.9y  education_high.school  \
0                   0                   0                      1   
1                   0       

In [5]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=automl_dataset,
    label_column_name='y',
    compute_target=compute_target,
    n_cross_validations=5)

In [6]:
# Submit your automl run
from azureml.widgets import RunDetails


### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting remote run.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_010aa149-434e-4c86-8554-4bcbcb2240a0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_010aa149-434e-4c86-8554-4bcbcb2240a0,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2982                          |1                               |26360                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_010aa149-434e-4c86-8554-4bcbcb2240a0',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-03-03T23:37:39.885772Z',
 'endTimeUtc': '2022-03-04T00:18:18.397945Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-187962","workspace_name":"quick-starts-ws-187962","region":"southcentralus","compute_target":"comput

In [8]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_metrics = automl_run.get_metrics()
print(automl_metrics)
print(automl_metrics)
best_run, model = automl_run.get_output()
print(automl_run.get_file_names())
# automl_run.download_file('outputs/model.pkl', './automl_model.pkl')

0.9163125948406676


['automl_driver.py', 'definition.json', 'definition_original.json', 'outputs/verifier_results.json']


In [ ]:
compute_target.delete()